In [6]:
import os
from dotenv import load_dotenv 
load_dotenv()
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [3]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
#Tracing and Tracking 
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")  
os.environ["LANGCHAIN_TRACING_V2"]= "ture"

In [4]:
class Product_detail(BaseModel):
    brand: str=Field(description="Brand of the product")
    model: str = Field(description="Model of the product")
    specs:str= Field(description="Specifications of the product")

class Product(BaseModel):
   prodect_name: str = Field(description="Name of the product")
   product_price: float = Field(description="Price of the product")
   product_details: Product_detail = Field(description="Details of the product")
   price: str = Field(description="Price of the product in string format")
   link: str = Field(description="URL Link to the product")

In [ ]:
#model
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-it", temperature=0.0)


In [7]:
#output parser
json_parser = JsonOutputParser(pydantic_object=Product)

In [9]:
promt = ChatPromptTemplate.from_messages([
    ("system", "You are an ecommerce search engine assistant who searches for requested products."),
    ("user", "Answer the user query strictly in the following JSON format without any additional text:\n{format_instructions}\nQuery: {query}")

]).partial(format_instructions=json_parser.get_format_instructions())

In [10]:
chain = promt | model | json_parser

In [16]:
try: 
    result =chain.invoke({"query": "Find me a laptop "})
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")
    result = None

{'prodect_name': 'Laptop', 'product_price': 1000.0, 'product_details': {'brand': 'Dell', 'model': 'XPS 13', 'specs': 'Intel Core i7, 16GB RAM, 512GB SSD'}, 'price': '$1000.00', 'link': 'https://www.example.com/dell-xps-13'}


In [17]:
result.get("product_name", "No product found")

'No product found'